In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
import re
import scipy.stats as st
import matplotlib.pyplot as plt


In [2]:
raw_data = pd.read_csv('all-weeks-countries.csv')
raw_data.head()

,country_name,country_iso2,week,category,weekly_rank,show_title,season_title,cumulative_weeks_in_top_10
0,Argentina,AR,2022-01-09,Films,1,Don't Look Up,NaN,3
1,Argentina,AR,2022-01-09,Films,2,Mother/Android,NaN,1
2,Argentina,AR,2022-01-09,Films,3,The Lost Daughter,NaN,2
3,Argentina,AR,2022-01-09,Films,4,Backtrace,NaN,1
4,Argentina,AR,2022-01-09,Films,5,Four to Dinner,NaN,1


In [3]:
raw_data['season_title'].unique()

array([nan, 'Café con aroma de mujer: Season 1', 'Cobra Kai: Season 4',
       'The Queen of Flow: Season 2', 'Stay Close: Limited Series',
       'The Queen of Flow: Season 1', 'Emily in Paris: Season 2',
       'Pasión de Gavilanes: Season 1', 'The Witcher: Season 2',
       'Rebelde: Season 1', 'Rebelde Way: Temporada 1',
       'Daughter From Another Mother: Season 2',
       'The Girl from Oslo: Season 1', 'Money Heist: Part 5',
       'The Witcher: Season 1', 'Yo soy Betty, la fea: Season 1',
       'Titans: Season 3', 'Pablo Escobar, el patrón del mal: Season 1',
       'Lost in Space: Season 3',
       'Jurassic World Camp Cretaceous: Season 4', 'Maid: Limited Series',
       'Elves: Season 1', 'Hellbound: Season 1',
       'True Story: Limited Series', 'Arcane: Season 1',
       'Squid Game: Season 1', 'You: Season 3',
       'Narcos: Mexico: Season 3', 'Luis Miguel - The Series: Season 3',
       'Where is Marta?: Limited Series', 'The Five Juanas: Season 1',
       'Maya and

In [4]:
raw_data.loc[:,'week'] = raw_data.loc[:,'week'].apply(lambda a: dt.strptime(a, '%Y-%m-%d'))

In [5]:
raw_data['Week No'] = raw_data['week'].apply(lambda a: dt.strftime(a, '%W'))
raw_data['Year'] = raw_data['week'].apply(lambda a: dt.strftime(a, '%Y'))
raw_data.head()

,country_name,country_iso2,week,category,weekly_rank,show_title,season_title,cumulative_weeks_in_top_10,Week No,Year
0,Argentina,AR,2022-01-09,Films,1,Don't Look Up,NaN,3,01,2022
1,Argentina,AR,2022-01-09,Films,2,Mother/Android,NaN,1,01,2022
2,Argentina,AR,2022-01-09,Films,3,The Lost Daughter,NaN,2,01,2022
3,Argentina,AR,2022-01-09,Films,4,Backtrace,NaN,1,01,2022
4,Argentina,AR,2022-01-09,Films,5,Four to Dinner,NaN,1,01,2022


In [6]:
raw_data['Week No'] = raw_data['Week No'].str.replace("01", "02")

In [7]:
raw_data['Week No'] = raw_data['Week No'].str.replace("00", "01")

In [8]:
films = raw_data[raw_data['season_title'].isnull()]
films.head()

,country_name,country_iso2,week,category,weekly_rank,show_title,season_title,cumulative_weeks_in_top_10,Week No,Year
0,Argentina,AR,2022-01-09,Films,1,Don't Look Up,NaN,3,02,2022
1,Argentina,AR,2022-01-09,Films,2,Mother/Android,NaN,1,02,2022
2,Argentina,AR,2022-01-09,Films,3,The Lost Daughter,NaN,2,02,2022
3,Argentina,AR,2022-01-09,Films,4,Backtrace,NaN,1,02,2022
4,Argentina,AR,2022-01-09,Films,5,Four to Dinner,NaN,1,02,2022


In [9]:
series = raw_data.dropna()
series.head()

,country_name,country_iso2,week,category,weekly_rank,show_title,season_title,cumulative_weeks_in_top_10,Week No,Year
10,Argentina,AR,2022-01-09,TV,1,Café con aroma de mujer,Café con aroma de mujer: Season 1,2,02,2022
11,Argentina,AR,2022-01-09,TV,2,Cobra Kai,Cobra Kai: Season 4,2,02,2022
12,Argentina,AR,2022-01-09,TV,3,The Queen of Flow,The Queen of Flow: Season 2,8,02,2022
13,Argentina,AR,2022-01-09,TV,4,Stay Close,Stay Close: Limited Series,1,02,2022
14,Argentina,AR,2022-01-09,TV,5,The Queen of Flow,The Queen of Flow: Season 1,10,02,2022


In [40]:
hh = pd.DataFrame(series.groupby('country_name')['weekly_rank'].unique())
hh

,weekly_rank
country_name,
Argentina,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]"
Australia,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]"
Austria,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]"
Bahamas,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]"
Bahrain,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]"
...,...
United Kingdom,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]"
United States,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]"
Uruguay,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]"


In [10]:
series.head(5000)

,country_name,country_iso2,week,category,weekly_rank,show_title,season_title,cumulative_weeks_in_top_10,Week No,Year
10,Argentina,AR,2022-01-09,TV,1,Café con aroma de mujer,Café con aroma de mujer: Season 1,2,02,2022
11,Argentina,AR,2022-01-09,TV,2,Cobra Kai,Cobra Kai: Season 4,2,02,2022
12,Argentina,AR,2022-01-09,TV,3,The Queen of Flow,The Queen of Flow: Season 2,8,02,2022
13,Argentina,AR,2022-01-09,TV,4,Stay Close,Stay Close: Limited Series,1,02,2022
14,Argentina,AR,2022-01-09,TV,5,The Queen of Flow,The Queen of Flow: Season 1,10,02,2022
...,...,...,...,...,...,...,...,...,...,...
10097,Dominican Republic,DO,2022-01-09,TV,8,The Witcher,The Witcher: Season 2,4,02,2022
10098,Dominican Republic,DO,2022-01-09,TV,9,Stay Close,Stay Close: Limited Series,1,02,2022
10099,Dominican Republic,DO,2022-01-09,TV,10,One-Punch Man,One-Punch Man: Season 2,2,02,2022
10110,Dominican Republic,DO,2022-01-02,TV,1,Café con aroma de mujer,Café con aroma de mujer: Season 1,1,01,2022


In [38]:
series.pivot_table(index = ['weekly_rank'], columns = ['show_title'], aggfunc = 'count', fill_value = 0)

Week No                                                \
show_title  60 Days In 7 Secretaries A Korean Odyssey A Love to Last   
weekly_rank                                                            
1                    0             0                0              0   
2                    0             0                0              0   
3                    0             0                0              0   
4                    0             0                0              0   
5                    0             0                0              3   
6                    0             0                0              0   
7                    0             1                1              6   
8                    0             0                0              4   
9                    1             0                0              2   
10                   0             0                0              3   

                                                                            \
show_title  A Sinister Sect: Colonia Dignidad A Tale Dark & Grimm A.P. Bio   
weekly_rank                                                                  
1                                           0                   0        0   
2                                           0                   0        0   
3                                           0                   0        0   
4                                           1                   0        0   
5                                           0                   2        0   
6                                           0                   4        1   
7                                           0                   1        0   
8                                           2                   4        0   
9                                           0                   7        0   
10                                          0                   6        2   

                                                              ...  \
show_title  Ackley Bridge Al-Mizan AlRawabi School for Girls  ...   
weekly_rank                                                   ...   
1                       0        0                        27  ...   
2                       0        0                        14  ...   
3                       0        0                        15  ...   
4                       0        2                        14  ...   
5                       0        1                        10  ...   
6                       0        2                        14  ...   
7                       0        2                        21  ...   
8                       0        2                         7  ...   
9                       1        2                        12  ...   
10                      0        0                         8  ...   

                            week                                      \
show_title  Yo soy Betty, la fea  You You Are My Spring Young Royals   
weekly_rank                                                            
1                             26   61                 0            1   
2                             62  123                 0            2   
3                             97  107                 1            1   
4                             68  103                 2            4   
5                             55   64                 3            3   
6                             33   79                 1            2   
7                             24  106                 1            1   
8                             21   93                 1            0   
9                             17  127                 2            2   
10                             9   87                 1            6   

                                                                   \
show_title  Young Sheldon Yu Yu Hakusho: Ghost Files ZeroZeroZero   
weekly_rank                                                         
1                    

In [30]:
series_country = pd.DataFrame(series.groupby('country_name')['season_title'].count())
series_country


,season_title
country_name,
Argentina,280
Australia,280
Austria,280
Bahamas,279
Bahrain,281
...,...
United Kingdom,280
United States,278
Uruguay,280


In [22]:
films_country = pd.DataFrame(films.groupby('country_name')['show_title'].count())
films_country

,show_title
country_name,
Argentina,280
Australia,280
Austria,280
Bahamas,281
Bahrain,279
...,...
United Kingdom,280
United States,282
Uruguay,280


In [32]:
countr_series = pd.DataFrame(series.groupby('show_title')['country_name'].count())
countr_series

,country_name
show_title,
60 Days In,1
7 Secretaries,1
A Korean Odyssey,1
A Love to Last,18
A Sinister Sect: Colonia Dignidad,3
...,...
Yu Yu Hakusho: Ghost Files,1
ZeroZeroZero,2
Zumbo's Just Desserts,1


In [35]:
x = []
for a in len(range(1,5001)):
    if a == series.iloc[4,a]:
             x.append(a)  
    else:
               pass

KeyError: 0

In [24]:
#season = pd.DataFrame(series.groupby(series["country_name"])["weekly_rank"].value_counts())
#season
#xyz = pd.DataFrame(series.groupby(["Week No", "Year"])["country_name"].count())
#xyz.head(20)